In [1]:
!pip install -q openai
!pip install -q datasets==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.0 MB/s eta 0:00:00


Downloading dataset

In [2]:
from datasets import load_dataset
import pickle
import pandas as pd
import time

In [3]:
dataset = load_dataset("AmazonScience/massive", name='fa-IR')
dataset = dataset.shuffle(seed=42)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset massive downloaded and prepared to /root/.cache/huggingface/datasets/AmazonScience___massive/fa-IR/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})

In [5]:
train = pd.DataFrame(dataset['train'])
train['intent']

0        16
1        45
2         4
3         4
4        52
         ..
11509    56
11510    32
11511    48
11512    32
11513    49
Name: intent, Length: 11514, dtype: int64

In [6]:
train = train[train['scenario'] == 3] #play subset
train['intent'].value_counts()

45    639
36    283
58    193
20    150
51    112
Name: intent, dtype: int64

In [7]:
train.head(100
           )

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
1,876,fa-IR,train,3,45,آهنگ خاطر خواه از ایوان بند را پخش کن,آهنگ خاطر خواه از [artist_name : ایوان بند] را...,3,"{'slot': ['artist_name'], 'method': ['localiza...","{'worker_id': ['21', '14', '8'], 'intent_score..."
17,11297,fa-IR,train,3,58,فایل صوتی آخرین ویدئو های یوگا را پخش کن,فایل صوتی آخرین [podcast_name : ویدئو های یوگا...,3,"{'slot': ['podcast_name'], 'method': ['transla...","{'worker_id': ['21', '8', '9'], 'intent_score'..."
32,6149,fa-IR,train,3,45,من میخوام موزیک ایوان بند از لیست موزیک خودم گ...,من میخوام موزیک ایوان بند از لیست موزیک خودم گ...,8,"{'slot': [], 'method': []}","{'worker_id': ['8', '14', '21'], 'intent_score..."
39,9223,fa-IR,train,3,36,رادیو ورزش را روشن کن,[radio_name : رادیو ورزش] را روشن کن,3,"{'slot': ['radio_name'], 'method': ['localizat...","{'worker_id': ['21', '14', '8'], 'intent_score..."
51,10328,fa-IR,train,3,51,بیا سر کندی کراش شرط ببندیم.,بیا سر [game_name : کندی کراش] شرط ببندیم.,16,"{'slot': ['game_name'], 'method': ['unchanged']}","{'worker_id': ['8', '14', '21'], 'intent_score..."
...,...,...,...,...,...,...,...,...,...,...
837,9463,fa-IR,train,3,36,بهترین از گروه آریان را بشنویم,بهترین از گروه آریان را بشنویم,13,"{'slot': [], 'method': []}","{'worker_id': ['14', '8', '3'], 'intent_score'..."
840,5213,fa-IR,train,3,45,من دوست دارم به موزیک هندوستانی گوش بدم,من دوست دارم به موزیک [music_genre : هندوستانی...,8,"{'slot': ['music_genre'], 'method': ['translat...","{'worker_id': ['14', '3', '8'], 'intent_score'..."
879,9623,fa-IR,train,3,36,برنامه اخبار را پخش کن,برنامه اخبار را پخش کن,3,"{'slot': [], 'method': []}","{'worker_id': ['8', '3', '21'], 'intent_score'..."
883,9240,fa-IR,train,3,36,میتونی رادیو جوان بدون هیچ تبلیغی پخش کنی,میتونی [radio_name : رادیو جوان] بدون هیچ تبلی...,8,"{'slot': ['radio_name'], 'method': ['localizat...","{'worker_id': ['3', '14', '8'], 'intent_score'..."


In [8]:
test = pd.DataFrame(dataset['test'])
test['intent']

0       50
1       45
2       52
3       52
4       50
        ..
2969    11
2970    13
2971    30
2972    36
2973    20
Name: intent, Length: 2974, dtype: int64

In [9]:
test.head()

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
0,7730,fa-IR,test,2,50,دوشنبه بعدی را علامت بزن,[date : دوشنبه] بعدی را علامت بزن,8,"{'slot': ['date'], 'method': ['translation']}","{'worker_id': ['8', '14', '3'], 'intent_score'..."
1,3492,fa-IR,test,3,45,لیست بیشترین آهنگ هایی که پخش شدند را برای من ...,لیست بیشترین آهنگ هایی که پخش شدند را برای من ...,8,"{'slot': [], 'method': []}","{'worker_id': ['8', '3', '14'], 'intent_score'..."
2,1205,fa-IR,test,16,52,زنگ هشدار را برای سه بعدازظهر غیرفعال کن,زنگ هشدار را برای [time : سه بعدازظهر] غیرفعال کن,8,"{'slot': ['time'], 'method': ['translation']}","{'worker_id': ['21', '3', '8'], 'intent_score'..."
3,2985,fa-IR,test,16,52,زنگ هشدار را کنسل کن,زنگ هشدار را کنسل کن,8,"{'slot': [], 'method': []}","{'worker_id': ['8', '3', '14'], 'intent_score'..."
4,6725,fa-IR,test,2,50,برنامه من برای بعد از ظهر را به من یادآوری کن,برنامه من برای [timeofday : بعد از ظهر] را به ...,8,"{'slot': ['timeofday'], 'method': ['translatio...","{'worker_id': ['3', '21', '8'], 'intent_score'..."


In [10]:
for i in range(18):
    print(i)
    print(len(test[test['scenario'] == i]))
    print(test[test['scenario'] == i]['intent'].value_counts())
    print('-----------------------------------------------------------')

0
106
47    81
27    25
Name: intent, dtype: int64
-----------------------------------------------------------
1
124
42    51
2     35
54    23
11    15
Name: intent, dtype: int64
-----------------------------------------------------------
2
402
50    209
32    126
30     67
Name: intent, dtype: int64
-----------------------------------------------------------
3
387
45    176
36     72
58     63
20     41
51     35
Name: intent, dtype: int64
-----------------------------------------------------------
4
124
22    124
Name: intent, dtype: int64
-----------------------------------------------------------
5
103
0     88
38    15
Name: intent, dtype: int64
-----------------------------------------------------------
6
94
6     43
19    31
55    20
Name: intent, dtype: int64
-----------------------------------------------------------
7
271
44    119
33    114
17     26
15     12
Name: intent, dtype: int64
-----------------------------------------------------------
8
220
40    43
56    36
1   

In [11]:
test = test[test['scenario'] == 3] #play subset
test['intent'].value_counts()

45    176
36     72
58     63
20     41
51     35
Name: intent, dtype: int64

In [12]:
n_test_sample = 1
test_prompt = test.groupby('intent').apply(lambda x: x.sample(n_test_sample))

test_prompt.head()

,,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
intent,,,,,,,,,,,
20,2052,9851,fa-IR,test,3,20,از جایی که آخرین بار رها کردم کتاب خسرو و شیری...,از جایی که آخرین بار رها کردم کتاب [audiobook_...,8,"{'slot': ['player_setting', 'audiobook_name'],...","{'worker_id': ['14', '21', '8'], 'intent_score..."
36,960,9562,fa-IR,test,3,36,میشه لطفا این برنامه را برایم پخش کنی,میشه لطفا این برنامه را برایم پخش کنی,8,"{'slot': [], 'method': []}","{'worker_id': ['14', '21', '8'], 'intent_score..."
45,2203,151,fa-IR,test,3,45,لطفا آن را دوباره پخش کن,لطفا آن را دوباره پخش کن,3,"{'slot': [], 'method': []}","{'worker_id': ['14', '21', '8'], 'intent_score..."
51,1851,10406,fa-IR,test,3,51,بازی شطرنج را برای خودم و خودت باز کن,بازی [game_name : شطرنج] را برای خودم و خودت ب...,8,"{'slot': ['game_name'], 'method': ['localizati...","{'worker_id': ['21', '14', '8'], 'intent_score..."
58,1430,11409,fa-IR,test,3,58,لطفا رد کن و به قسمت بعدی فایل صوتی برو,لطفا رد کن و به [podcast_descriptor : قسمت بعد...,8,"{'slot': ['podcast_descriptor'], 'method': ['t...","{'worker_id': ['3', '8', '14'], 'intent_score'..."


In [13]:
n_sample_each_group = 3
demonstration = train.groupby('intent').apply(lambda x: x.sample(n_sample_each_group))
demonstration = demonstration.reset_index(drop=True)
demonstration = demonstration.sample(frac=1).reset_index(drop=True)

demonstration.head()

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
0,9607,fa-IR,train,3,36,ترس را پخش کن,ترس را پخش کن,8,"{'slot': [], 'method': []}","{'worker_id': ['8', '14', '3'], 'intent_score'..."
1,9719,fa-IR,train,3,20,برگرد به دا,[player_setting : برگرد به] [audiobook_name : دا],8,"{'slot': ['player_setting', 'audiobook_name'],...","{'worker_id': ['14', '21', '8'], 'intent_score..."
2,11382,fa-IR,train,3,58,در ستایش ترس قسمت بیست و سوم را پخش کن,[podcast_name : در ستایش ترس] قسمت بیست و سوم ...,8,"{'slot': ['podcast_name'], 'method': ['localiz...","{'worker_id': ['3', '8', '14'], 'intent_score'..."
3,5626,fa-IR,train,3,45,برای من آهنگ از شادمهر پخش کن,برای من آهنگ از [artist_name : شادمهر] پخش کن,8,"{'slot': ['artist_name'], 'method': ['localiza...","{'worker_id': ['3', '14', '8'], 'intent_score'..."
4,2481,fa-IR,train,3,45,لیست آهنگ ها را باز کن,لیست آهنگ ها را باز کن,3,"{'slot': [], 'method': []}","{'worker_id': ['8', '3', '21'], 'intent_score'..."


In [14]:
intent_map = {45: 'play_music', 36: 'play_radio', 58:'play_podcasts', 20:'play_audiobook', 51:'play_game'}

OpenAI prompting

In [ ]:
import openai
openai.api_key  = ""

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]

In [ ]:
max_retries = 5

### Zero-shot

In [ ]:
prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

utterance: SAMPLE
intent:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")


An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: play_audiobook
Actual Label: play_audiobook
---------------------------------------------------------------------
utterance: الان رادیو روشن کن
Model Response: play_radio
Actual Label: play_radio
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to ha

In [ ]:
prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

جمله: SAMPLE
نیت:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")


utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: play_audiobook
Actual Label: play_audiobook
---------------------------------------------------------------------
utterance: الان رادیو روشن کن
Model Response: play_radio
Actual Label: play_radio
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: لیست پخش جز تو من را پخش کن
Model Response: play_music
Actual Label: play_music
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4

In [ ]:
intent_map = {45: 'پخش_موسیقی', 36: 'پخش_رادیو', 58:'پخش_پادکست', 20:'پخش_کتاب_صوتی', 51:'بازی_کردن'}

In [ ]:
prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: پخش_موسیقی, پخش_رادیو, پخش_پادکست, پخش_کتاب_صوتی, بازی_کردن. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

جمله: SAMPLE
نیت:
"""


for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")


utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: پخش_کتاب_صوتی
Actual Label: پخش_کتاب_صوتی
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: الان رادیو روشن کن
Model Response: پخش_رادیو
Actual Label: پخش_رادیو
---------------------------------------------------------------------
utterance: لیست پخش جز تو من را پخش کن
Model Response: پخش_موسیقی
Actual Label: پخش_موسیقی
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcY

In [ ]:
acceptable_classes = [
    "play_music",
    "play_radio",
    "play_podcasts",
    "play_audiobook",
    "play_game",
]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=1)
    precision = precision_score(actual, predicted, average='weighted', zero_division=1)
    recall = recall_score(actual, predicted, average='weighted', zero_division=1)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = set(actual + predicted)

    metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'Class-wise Counts': {}
    }

    for class_label in classes:
        class_precision = precision_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_recall = recall_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_f1 = f1_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)

        metrics['Class-wise Counts'][class_label] = {
            'Actual Count': actual_counts[class_label],
            'Predicted Count': predicted_counts[class_label],
            'Precision': class_precision,
            'Recall': class_recall,
            'F1 Score': class_f1
        }

    for metric, value in metrics.items():
        if metric == 'Class-wise Counts':
            print(f"\n{metric}:")
            for class_label, stats in value.items():
                print(f"Class: {class_label}")
                for stat, val in stats.items():
                    print(f"{stat}: {val}")
                print()
        else:
            print(f"{metric}: {value}")

    return metrics

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

utterance: SAMPLE
intent:
"""
c = 0
for index, row in tqdm(test.iterrows(), total=len(test)):
    # c += 1
    # if (c == 2):
    #     time.sleep(30)
    #     c = 0
    retries = 0
    while retries < max_retries:
        # time.sleep(20)
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            actual.append(intent_map[row['intent']])
            if response not in acceptable_classes:
                print(prompt.replace("SAMPLE", row['utt']), response)

            predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    time.sleep(20)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

  4%|▍         | 15/387 [05:08<2:07:13, 20.52s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 None of the above (This is not a valid intent for a virtual assistant)


 40%|████      | 156/387 [53:13<1:18:54, 20.50s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 play_previous_song


 65%|██████▍   | 250/387 [1:25:18<46:46, 20.49s/it]

...

 68%|██████▊   | 264/387 [1:30:05<42:04, 20.53s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)


 86%|████████▋ | 334/387 [1:54:01<18:06, 20.51s/it]


The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) 
utterance: SAMPLE
intent:
 upload_audiobooks


100%|██████████| 387/387 [2:12:08<00:00, 20.49s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.7700258397932817
F1 Score: 0.7517118450141267
Precision: 0.7925609543559198
Recall: 0.7700258397932817

Class-wise Counts:
Class: None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: None of the above (This is not a valid intent for a virtual assistant)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: stop_playing
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: stop_audiobook
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: play_music
Actual Count: 176
Predicted Count: 189
Precision: 0.8835978835978836
Recall: 0.9488636363636364
F1 Score: 0.915068493150685

Class: play_podcasts
Actual Count: 63
Predicted Count: 15
Precision: 0.6666666666666666
Recall: 0.15873015873015872
F1 Score: 0.2564102564102564

Class: play_game
Actual Count: 35
Pre

{'Accuracy': 0.7700258397932817,
 'F1 Score': 0.7517118450141267,
 'Precision': 0.7925609543559198,
 'Recall': 0.7700258397932817,
 'Class-wise Counts': {'None of the above (The utterance translates to "Stop" and does not indicate any of the listed intents.)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'None of the above (This is not a valid intent for a virtual assistant)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'stop_playing': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'stop_audiobook': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'play_music': {'Actual Count': 176,
   'Predicted Count': 189,
   'Precision': 0.8835978835978836,
   'Recall': 0.9488636363636364,
   'F1 Score': 0.915068493150685},
  'play_podcasts': {'Actual Count': 

In [ ]:
from tqdm import tqdm

actual = []
predicted = []


prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

جمله: SAMPLE
نیت:
"""
c = 0
for index, row in tqdm(test.iterrows(), total=len(test)):
    # c += 1
    # if (c == 2):
    #     time.sleep(30)
    #     c = 0
    retries = 0
    while retries < max_retries:
        # time.sleep(20)
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            actual.append(intent_map[row['intent']])
            if response not in acceptable_classes:
                print(prompt.replace("SAMPLE", row['utt']), response)

            predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    time.sleep(20)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

  0%|          | 1/387 [00:21<2:18:00, 21.45s/it]


جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) 
جمله: پخش کتاب را متوقف کن
نیت:
 stop_audiobook


  1%|          | 3/387 [01:05<2:20:17, 21.92s/it]

An error occurred: The server is overloaded or not ready yet.
Retrying (1/5)...


  4%|▍         | 17/387 [07:00<2:09:27, 20.99s/it]

...

 95%|█████████▌| 368/387 [2:13:25<06:40, 21.06s/it]


جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) 
جمله: شطرنج بازی کن
نیت:
 play_game (بازی کردن)


 98%|█████████▊| 379/387 [2:17:15<02:47, 20.95s/it]


جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) 
جمله: لطفا بازی شطرنج را باز کن و با من بازی کن
نیت:
 play_game (بازی شطرنج)


100%|██████████| 387/387 [2:20:01<00:00, 21.71s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.7467700258397932
F1 Score: 0.7420299492556893
Precision: 0.831843795527058
Recall: 0.7467700258397932

Class-wise Counts:
Class: play_music
Actual Count: 176
Predicted Count: 192
Precision: 0.8802083333333334
Recall: 0.9602272727272727
F1 Score: 0.9184782608695653

Class: آپلود کردن فایل های صوتی.
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: play_audiobook
Actual Count: 41
Predicted Count: 66
Precision: 0.4090909090909091
Recall: 0.6585365853658537
F1 Score: 0.5046728971962617

Class: play_music (پخش آهنگ های غمگین راک)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: پخش کتاب صوتی عاشقانه (play_audiobook)
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: باز کردن پوشه گروه بمرانی.
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: play_game
Actual Count: 35
Predicted Count: 24
Precision: 0.875
Recall: 0.6
F1 Score: 0.711864406779661



{'Accuracy': 0.7467700258397932,
 'F1 Score': 0.7420299492556893,
 'Precision': 0.831843795527058,
 'Recall': 0.7467700258397932,
 'Class-wise Counts': {'play_music': {'Actual Count': 176,
   'Predicted Count': 192,
   'Precision': 0.8802083333333334,
   'Recall': 0.9602272727272727,
   'F1 Score': 0.9184782608695653},
  'آپلود کردن فایل های صوتی.': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'play_audiobook': {'Actual Count': 41,
   'Predicted Count': 66,
   'Precision': 0.4090909090909091,
   'Recall': 0.6585365853658537,
   'F1 Score': 0.5046728971962617},
  'play_music (پخش آهنگ های غمگین راک)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'پخش کتاب صوتی عاشقانه (play_audiobook)': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'باز کردن پوشه گروه بمرانی.': {'Actual Count': 0,
   'Predicted Count': 1,

In [ ]:
intent_map = {45: 'پخش_موسیقی', 36: 'پخش_رادیو', 58:'پخش_پادکست', 20:'پخش_کتاب_صوتی', 51:'بازی_کردن'}

In [ ]:
acceptable_classes = [
    "پخش_موسیقی",
    "پخش_رادیو",
    "پخش_پادکست",
    "پخش_کتاب_صوتی",
    "بازی_کردن",
]

In [ ]:
from tqdm import tqdm

actual = []
predicted = []


prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: پخش_موسیقی, پخش_رادیو, پخش_پادکست, پخش_کتاب_صوتی, بازی_کردن. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

جمله: SAMPLE
نیت:
"""
c = 0
for index, row in tqdm(test.iterrows(), total=len(test)):
    # c += 1
    # if (c == 2):
    #     time.sleep(30)
    #     c = 0
    retries = 0
    while retries < max_retries:
        # time.sleep(20)
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            actual.append(intent_map[row['intent']])
            if response not in acceptable_classes:
                print(prompt.replace("SAMPLE", row['utt']), response)

            predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    time.sleep(20)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

  3%|▎         | 10/387 [03:34<2:15:16, 21.53s/it]


جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: پخش_موسیقی, پخش_رادیو, پخش_پادکست, پخش_کتاب_صوتی, بازی_کردن. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) 
جمله: بازی ها را باز کن
نیت:
 بازی کردن


  4%|▎         | 14/387 [04:58<2:12:13, 21.27s/it]


جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: پخش_موسیقی, پخش_رادیو, پخش_پادکست, پخش_کتاب_صوتی, بازی_کردن. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) 
جمله: بازی کن
نیت:
 بازی کردن


  7%|▋         | 27/387 [09:35<2:08:39, 21.44s/it]

...

 93%|█████████▎| 361/387 [2:37:48<09:01, 20.84s/it]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 29 Jun 2023 17:12:16 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7defc38b284c0b89-AMS', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


 98%|█████████▊| 378/387 [2:49:24<03:10, 21.17s/it]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 29 Jun 2023 17:23:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7defd489e82bb97b-AMS', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


100%|██████████| 387/387 [2:58:18<00:00, 27.65s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.7364341085271318
F1 Score: 0.7248972226064164
Precision: 0.8130073343251637
Recall: 0.7364341085271318

Class-wise Counts:
Class: بازی_کردن
Actual Count: 35
Predicted Count: 20
Precision: 0.8
Recall: 0.45714285714285713
F1 Score: 0.5818181818181818

Class: پخش_پادکست
Actual Count: 63
Predicted Count: 12
Precision: 0.9166666666666666
Recall: 0.1746031746031746
F1 Score: 0.29333333333333333

Class: پخش_کتاب_صوتی
Actual Count: 41
Predicted Count: 66
Precision: 0.4090909090909091
Recall: 0.6585365853658537
F1 Score: 0.5046728971962617

Class: پخش_موسیقی
Actual Count: 176
Predicted Count: 198
Precision: 0.8585858585858586
Recall: 0.9659090909090909
F1 Score: 0.9090909090909091

Class: آپلود کردن تمام فایل های صوتی فرد
Actual Count: 0
Predicted Count: 1
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: بازی کردن
Actual Count: 0
Predicted Count: 18
Precision: 0.0
Recall: 1.0
F1 Score: 0.0

Class: پخش_رادیو
Actual Count: 72
Predicted Count: 72
Precision: 0.8472222222222222
Recall: 0

{'Accuracy': 0.7364341085271318,
 'F1 Score': 0.7248972226064164,
 'Precision': 0.8130073343251637,
 'Recall': 0.7364341085271318,
 'Class-wise Counts': {'بازی_کردن': {'Actual Count': 35,
   'Predicted Count': 20,
   'Precision': 0.8,
   'Recall': 0.45714285714285713,
   'F1 Score': 0.5818181818181818},
  'پخش_پادکست': {'Actual Count': 63,
   'Predicted Count': 12,
   'Precision': 0.9166666666666666,
   'Recall': 0.1746031746031746,
   'F1 Score': 0.29333333333333333},
  'پخش_کتاب_صوتی': {'Actual Count': 41,
   'Predicted Count': 66,
   'Precision': 0.4090909090909091,
   'Recall': 0.6585365853658537,
   'F1 Score': 0.5046728971962617},
  'پخش_موسیقی': {'Actual Count': 176,
   'Predicted Count': 198,
   'Precision': 0.8585858585858586,
   'Recall': 0.9659090909090909,
   'F1 Score': 0.9090909090909091},
  'آپلود کردن تمام فایل های صوتی فرد': {'Actual Count': 0,
   'Predicted Count': 1,
   'Precision': 0.0,
   'Recall': 1.0,
   'F1 Score': 0.0},
  'بازی کردن': {'Actual Count': 0,
   'Pr

### Few-shot

In [ ]:
intent_map = {45: 'play_music', 36: 'play_radio', 58:'play_podcasts', 20:'play_audiobook', 51:'play_game'}

In [ ]:
def create_demonstration_prompt(demonstration_df, title="Example",
                                input="Utterance", label="Intent",
                                label_map=None):
  result = ""
  i = 1
  for index, row in demonstration_df.iterrows():
    result += f"{title} {i}:\n"
    result += f"{input}: {row['utt']}\n"
    if label_map is None:
      result += f"{label}: {row['intent']}\n"
    else:
      result += f"{label}: {label_map[row['intent']]}\n"
    i += 1
  return result

In [ ]:
print(create_demonstration_prompt(demonstration, label_map=intent_map))

Example 1:
Utterance: آهنگ لیلا را پخش کن
Intent: play_music
Example 2:
Utterance: ترس از شادمهر را پخش کن
Intent: play_music
Example 3:
Utterance: به قسمت بعدی این فایل صوتی برو
Intent: play_podcasts
Example 4:
Utterance: از تنظیمات زمان استفاده کن
Intent: play_audiobook
Example 5:
Utterance: دا را پخش کن
Intent: play_audiobook
Example 6:
Utterance: من دوست دارم الان jeopardy بازی کنیم
Intent: play_game
Example 7:
Utterance: با من جورچین بازی کن
Intent: play_game
Example 8:
Utterance: لطفا رادیو ورزش بزار
Intent: play_radio
Example 9:
Utterance: میشه لطفا رادیو ورزش را در جایی که من هستم روشن کنی
Intent: play_radio
Example 10:
Utterance: لطفا بهترین آهنگ من را پخش کن
Intent: play_music
Example 11:
Utterance: قسمت بعد از فایل صوتی مونولوگ
Intent: play_podcasts
Example 12:
Utterance: من میخوام به آهنگ های برتر امروز گوش بدم
Intent: play_radio
Example 13:
Utterance: قسمت بعدی پادکست مونولوگ
Intent: play_podcasts
Example 14:
Utterance: ادامه ی کتاب صوتی تاریخ روم رو پخش کن
Intent: play_au

In [ ]:
prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

Examples:
{create_demonstration_prompt(demonstration, label_map=intent_map)}

utterance: SAMPLE
intent:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            # print(prompt.replace("SAMPLE", row['utt']), response)
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: play_audiobook
Actual Label: play_audiobook
---------------------------------------------------------------------
utterance: الان رادیو روشن کن
Model Response: play_radio
Actual Label: play_radio
---------------------------------------------------------------------
utterance: لیست پخش جز تو من را پخش کن
Model Response: play_music
Actual Label: play_music
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: لطفا جنگ قبیله‌ها را بازی کن
Model Response: play_game
Actual Label: play_

In [ ]:
prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: play_music, play_radio, play_podcasts, play_audiobook, play_game. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

مثالها:
{create_demonstration_prompt(demonstration, label_map=intent_map)}

جمله: SAMPLE
نیت:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            # print(prompt.replace("SAMPLE", row['utt']), response)
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: play_audiobook
Actual Label: play_audiobook
---------------------------------------------------------------------
utterance: الان رادیو روشن کن
Model Response: play_radio
Actual Label: play_radio
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
utterance: لیست پخش جز تو من را پخش کن
Model Response: play_music
Actual Label: play_music
---------------------------------------------------------------------
utterance: لطفا جنگ قبیله‌ها را بازی کن
Model Response: play_game
Actual Label: play_

In [ ]:
intent_map = {45: 'پخش_موسیقی', 36: 'پخش_رادیو', 58:'پخش_پادکست', 20:'پخش_کتاب_صوتی', 51:'بازی_کردن'}

In [ ]:
prompt = f"""
جمله زیر صحبت یک فرد با یک دستیار مجازی است. هدف (نیت) مرتبط با هر جمله را تعیین کن.
پاسخ های ممکن عبارتند از: پخش_موسیقی, پخش_رادیو, پخش_پادکست, پخش_کتاب_صوتی, بازی_کردن. (لطفاً فقط یکی از این موارد را پاسخ بده و نه چیز دیگری!) \

مثالها:
{create_demonstration_prompt(demonstration, label_map=intent_map)}

جمله: SAMPLE
نیت:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            # print(prompt.replace("SAMPLE", row['utt']), response)
            print(f"utterance: {row['utt']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {intent_map[row['intent']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

utterance: کتاب صوتی افسانه های ایرانی را پخش کن
Model Response: پخش_کتاب_صوتی
Actual Label: پخش_کتاب_صوتی
---------------------------------------------------------------------
utterance: الان رادیو روشن کن
Model Response: پخش_رادیو
Actual Label: پخش_رادیو
---------------------------------------------------------------------
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
An error occurred: The server is overloaded or not ready yet.
Retrying (2/5)...
utterance: لیست پخش جز تو من را پخش کن
Model Response: پخش_موسیقی
Actual Label: پخش_موسیقی
---------------------------------------------------------------------
utterance

In [ ]:
acceptable_classes = [
    "play_music",
    "play_radio",
    "play_podcasts",
    "play_audiobook",
    "play_game",
]

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's utterance to a virtual assistant. The utterance is in Persian. Identify the intent associated with it. \
Possible answers are: play_music, play_radio, play_podcasts, play_audiobook, play_game. (Please only reply one of these and nothing else!) \

Examples:
{create_demonstration_prompt(demonstration, label_map=intent_map)}

utterance: SAMPLE
intent:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE", row['utt']))
            actual.append(intent_map[row['intent']])

            if response not in acceptable_classes:
              print(prompt.replace("SAMPLE", row['utt']), response)

            predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)
    time.sleep(20)

    if retries == max_retries:
        print(f"Failed to process sample after {max_retries} retries. Skipping to the next sample.")

 38%|███▊      | 147/387 [50:17<1:21:56, 20.48s/it]

An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (2/5)...
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day

 38%|███▊      | 148/387 [53:07<4:20:59, 65.52s/it]

Failed to process sample after 5 retries. Skipping to the next sample.
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (2/5)...
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organiza

 39%|███▊      | 149/387 [54:58<5:13:37, 79.06s/it]

An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (1/5)...
An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-4mvcYVBwmYmrtxYauFgReUJz on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.
Retrying (2/5)...


 39%|███▊      | 149/387 [55:52<1:29:14, 22.50s/it]


KeyboardInterrupt: ignored

In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.8513513513513513
F1 Score: 0.8453814957174419
Precision: 0.8490939803439804
Recall: 0.8513513513513513

Class-wise Counts:
Class: play_game
Actual Count: 13
Predicted Count: 13
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class: play_music
Actual Count: 66
Predicted Count: 75
Precision: 0.8666666666666667
Recall: 0.9848484848484849
F1 Score: 0.9219858156028369

Class: play_podcasts
Actual Count: 22
Predicted Count: 16
Precision: 0.8125
Recall: 0.5909090909090909
F1 Score: 0.6842105263157896

Class: play_radio
Actual Count: 25
Predicted Count: 22
Precision: 0.8636363636363636
Recall: 0.76
F1 Score: 0.8085106382978724

Class: play_audiobook
Actual Count: 22
Predicted Count: 22
Precision: 0.7272727272727273
Recall: 0.7272727272727273
F1 Score: 0.7272727272727273



{'Accuracy': 0.8513513513513513,
 'F1 Score': 0.8453814957174419,
 'Precision': 0.8490939803439804,
 'Recall': 0.8513513513513513,
 'Class-wise Counts': {'play_game': {'Actual Count': 13,
   'Predicted Count': 13,
   'Precision': 1.0,
   'Recall': 1.0,
   'F1 Score': 1.0},
  'play_music': {'Actual Count': 66,
   'Predicted Count': 75,
   'Precision': 0.8666666666666667,
   'Recall': 0.9848484848484849,
   'F1 Score': 0.9219858156028369},
  'play_podcasts': {'Actual Count': 22,
   'Predicted Count': 16,
   'Precision': 0.8125,
   'Recall': 0.5909090909090909,
   'F1 Score': 0.6842105263157896},
  'play_radio': {'Actual Count': 25,
   'Predicted Count': 22,
   'Precision': 0.8636363636363636,
   'Recall': 0.76,
   'F1 Score': 0.8085106382978724},
  'play_audiobook': {'Actual Count': 22,
   'Predicted Count': 22,
   'Precision': 0.7272727272727273,
   'Recall': 0.7272727272727273,
   'F1 Score': 0.7272727272727273}}}

In [ ]:
# import pickle

# with open('actual.pkl','wb') as f:
#     pickle.dump(actual, f)

In [ ]:
# with open('predicted.pkl','wb') as f:
#     pickle.dump(predicted, f)